# Kubernetes Pods

In [57]:
# Cleanup

kubectl get all
#kubectl delete po/2-containers-in-a-pod
#kubectl delete deploy k8s-demo
#kubectl delete service k8s-demo-service

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


We've seen already how we can interrogate our cluster server using the kubectl tool, obtaining information about objects such as some system pods, in the system.

Now let's create our own pods using kubectl.

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
    
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

In [58]:
echo; echo "-- initial kubectl get all"
kubectl get all

echo; echo "-- kubectl run k8s0demo"
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080

echo; echo "-- second kubectl get all"
kubectl get all


-- initial kubectl get all
NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h

-- kubectl run k8s0demo
deployment "k8s-demo" created

-- second kubectl get all
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         0         1s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         0         1s

NAME                           READY     STATUS              RESTARTS   AGE
po/k8s-demo-7bc74f7946-6bst7   0/1       ContainerCreating   0          1s
po/k8s-demo-7bc74f7946-bkfhk   0/1       ContainerCreating   0          1s

NAME             TY

In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

In [59]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           14s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         14s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           14s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         14s

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-7bc74f7946-6bst7   1/1       Running   0          14s
po/k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          14s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


... with more information, using option '*-o wide*'

In [60]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           26s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-7bc74f7946   2         2         2         26s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=3673093502

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           26s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-7bc74f7946   2         2         2         26s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=3673093502

NAME              

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [61]:
kubectl get deploy/k8s-demo -o wide

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
k8s-demo   2         2         2            2           33s       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo


we get ...

In [62]:
kubectl get deploy/k8s-demo -o yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-01-19T14:16:37Z
  generation: 1
  labels:
    app: k8s-demo
  name: k8s-demo
  namespace: default
  resourceVersion: "111146"
  selfLink: /apis/extensions/v1beta1/namespaces/default/deployments/k8s-demo
  uid: 5c7a908c-fd23-11e7-8b3e-02420ac00002
spec:
  replicas: 2
  selector:
    matchLabels:
      app: k8s-demo
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: k8s-demo
    spec:
      containers:
      - image: mjbright/k8s-demo:1
        imagePullPolicy: IfNotPresent
        name: k8s-demo
        ports:
        - containerPort: 8080
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPol

In [72]:
cat examples/3.pod_example.yaml


apiVersion: v1
kind: Pod
metadata:
  name: 2-containers-in-a-pod
spec:
  containers:
  - name: k8s-demo
    image: mjbright/k8s-demo:1
    ports:
    - containerPort: 9876
  - name: shell
    image: alpine
    command:
      - "bin/sh"
      - "-c"
      - "while true; do date; sleep 1; done"



In [73]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           3m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         3m

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           3m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         3m

NAME                           READY     STATUS    RESTARTS   AGE
po/2-containers-in-a-pod       2/2       Running   0          1m
po/k8s-demo-7bc74f7946-6bst7   1/1       Running   0          3m
po/k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          3m

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


In [76]:
#kubectl delete po/2-containers-in-a-pod
#kubectl get pods

NAME                        READY     STATUS        RESTARTS   AGE
2-containers-in-a-pod       2/2       Terminating   0          2m
k8s-demo-7bc74f7946-6bst7   1/1       Running       0          3m
k8s-demo-7bc74f7946-bkfhk   1/1       Running       0          3m


In [77]:
kubectl create -f examples/3.pod_example.yaml

pod "2-containers-in-a-pod" created


In [78]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           7m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         7m

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           7m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         7m

NAME                           READY     STATUS    RESTARTS   AGE
po/2-containers-in-a-pod       2/2       Running   0          8s
po/k8s-demo-7bc74f7946-6bst7   1/1       Running   0          7m
po/k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          7m

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


In [79]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           7m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         7m

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           7m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         7m

NAME                           READY     STATUS    RESTARTS   AGE
po/2-containers-in-a-pod       2/2       Running   0          11s
po/k8s-demo-7bc74f7946-6bst7   1/1       Running   0          7m
po/k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          7m

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22h


In [80]:
kubectl describe po/2-containers-in-a-pod

Name:         2-containers-in-a-pod
Namespace:    default
Node:         kube-node-2/10.192.0.4
Start Time:   Fri, 19 Jan 2018 14:23:36 +0000
Labels:       <none>
Annotations:  <none>
Status:       Running
IP:           10.192.3.6
Containers:
  k8s-demo:
    Container ID:   docker://9c2a62ca441b6863facafb429456d56e923cd264e31544633ab2b0d354cd5210
    Image:          mjbright/k8s-demo:1
    Image ID:       docker-pullable://mjbright/k8s-demo@sha256:c5790558dfb46014b1d3eae68d33998b411a4f3f3ed476dd169f3f00491d923b
    Port:           9876/TCP
    State:          Running
      Started:      Fri, 19 Jan 2018 14:23:38 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-lpmff (ro)
  shell:
    Container ID:  docker://6fe0f0b1d622691e4d209b0c77e02c5deec780d40368487c28fa26e67f9a907e
    Image:         alpine
    Image ID:      docker-pullable://alpine@sha256:7df6db5aa61ae9480f52f0b3a06a

In [86]:
kubectl exec -it 2-containers-in-a-pod -n default echo hello world

Defaulting container name to k8s-demo.
Use 'kubectl describe pod/2-containers-in-a-pod -n default' to see all of the containers in this pod.
hello world


In [90]:
kubectl logs 2-containers-in-a-pod shell | head -3

Fri Jan 19 14:23:39 UTC 2018
Fri Jan 19 14:23:40 UTC 2018
Fri Jan 19 14:23:41 UTC 2018


In [89]:
kubectl delete po/2-containers-in-a-pod

pod "2-containers-in-a-pod" deleted


In [92]:
kubectl get pods

NAME                        READY     STATUS    RESTARTS   AGE
k8s-demo-7bc74f7946-6bst7   1/1       Running   0          10m
k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          10m


## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [93]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-7bc74f7946-6bst7   1/1       Running   0          10m       10.192.3.5   kube-node-2
k8s-demo-7bc74f7946-bkfhk   1/1       Running   0          10m       10.192.2.4   kube-node-1


In [94]:
POD1=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $1;}')
POD2=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1=k8s-demo-7bc74f7946-6bst7 POD1_IP=10.192.3.5
POD2=k8s-demo-7bc74f7946-bkfhk POD2_IP=10.192.2.4


Let's look at the yaml information for POD1:

In [95]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/created-by: |
      {"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"ReplicaSet","namespace":"default","name":"k8s-demo-7bc74f7946","uid":"5c7be8fb-fd23-11e7-8b3e-02420ac00002","apiVersion":"extensions","resourceVersion":"111110"}}
  creationTimestamp: 2018-01-19T14:16:37Z
  generateName: k8s-demo-7bc74f7946-
  labels:
    app: k8s-demo
    pod-template-hash: "3673093502"
  name: k8s-demo-7bc74f7946-6bst7
  namespace: default
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: k8s-demo-7bc74f7946
    uid: 5c7be8fb-fd23-11e7-8b3e-02420ac00002
  resourceVersion: "111140"
  selfLink: /api/v1/namespaces/default/pods/k8s-demo-7bc74f7946-6bst7
  uid: 5c84120e-fd23-11e7-8b3e-02420ac00002
spec:
  containers:
  - image: mjbright/k8s-demo:1
    imagePullPolicy: IfNotPresent
    name: k8s-demo
    ports:
    - containerPort: 808

So now we can access to the pods directly on their port 8080:

In [96]:
curl http://$POD1_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-7bc74f7946-6bst7


In [97]:
curl http://$POD2_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-7bc74f7946-bkfhk


# Kube-proxy

## TODO: Accessing our application - accessing via kube-proxy

<hr/>